In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [3]:
class Network(object):

    def __init__(self, sizes, func_model):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.activate_func = func_model["activate_func"]
        self.activate_func_prime = func_model["activate_func_prime"]
        self.cost_func = func_model["cost_func"]
        self.cost_derivative = func_model["cost_derivative"]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = self.activate_func(np.dot(w, a) + b)
        return a


    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        training_data = list(training_data)
        n = len(training_data)
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k + mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} Cost: {}".format(
                    j, self.evaluate(test_data)))
            else:
                print("Epoch {} complete".format(j))


    def evaluate(self,test_data):
        return np.array([self.cost_func(self.feedforward(item[0]),item[1]) for item in test_data]).mean()

    def update_mini_batch(self, mini_batch, eta):

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = self.activate_func(z)
            activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) * \
            self.activate_func_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.activate_func_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)

In [4]:
data=pd.read_excel('training_data.xlsx')

In [5]:
print(data.columns)

Index(['buffer', 'RTT', 'avg_speed', 'buffer_delay', 'stuck_ratio'], dtype='object')


In [5]:
for item in data.columns:
    print(data[item].describe())

count     74136.000000
mean      40644.076751
std       22525.913360
min         304.000000
25%       22718.000000
50%       37189.500000
75%       55428.250000
max      159225.000000
Name: buffer, dtype: float64
count    74136.000000
mean        51.597119
std         21.781843
min         15.000000
25%         36.000000
50%         47.000000
75%         61.000000
max        149.000000
Name: RTT, dtype: float64
count    74136.00000
mean      5739.62902
std        968.04202
min         10.00000
25%       5644.00000
50%       5918.00000
75%       6147.00000
max      29332.00000
Name: avg_speed, dtype: float64
count    74136.000000
mean      1440.801932
std       1383.820098
min        510.000000
25%        855.000000
50%       1094.000000
75%       1557.000000
max      30552.000000
Name: buffer_delay, dtype: float64
count    74136.000000
mean         0.056414
std          0.790739
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         44.66700

In [92]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))


def cost_derivative(output_activations, y):
    # return (output_activations-y)


def cost_func(output_activations, y):
    # return 0.5 * np.linalg.norm(output_activations - y)


func_model = {"activate_func": sigmoid,
              "activate_func_prime": sigmoid_prime,
              "cost_func": cost_func,
              "cost_derivative": cost_derivative, }

In [113]:
training_data=data[data['stuck_ratio']!=0]
training_data.head()

,buffer,RTT,avg_speed,buffer_delay,stuck_ratio
138,10575,36,1643,4665,1.355
206,40323,73,5844,2817,0.025
249,22609,27,6792,5990,0.170
262,22648,94,3105,1879,0.299
501,7462,58,6262,3193,0.029


In [114]:
training_data.loc[:,'buffer']/=5000
training_data.loc[:,'avg_speed']/=200
training_data.head()

C:\Users\84338\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,buffer,RTT,avg_speed,buffer_delay,stuck_ratio
138,2.1150,36,8.215,4665,1.355
206,8.0646,73,29.220,2817,0.025
249,4.5218,27,33.960,5990,0.170
262,4.5296,94,15.525,1879,0.299
501,1.4924,58,31.310,3193,0.029


In [81]:
training_data['buffer_delay']/=300

C:\Users\84338\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [82]:
inputs=np.vstack([training_data["buffer"].values,training_data["RTT"].values,training_data["avg_speed"].values]).transpose()

inputs=np.hsplit(inputs.transpose(),inputs[:,0].size)

outputs=np.vstack([training_data["buffer_delay"].values,training_data["stuck_ratio"].values]).transpose()

outputs=np.hsplit(outputs.transpose(),outputs[:,0].size)

In [83]:
_train=list(zip(inputs,outputs))
type(_train[0][1])
_train[0]

(array([[  2.115],
        [ 36.   ],
        [  8.215]]), array([[ 15.55 ],
        [  1.355]]))

In [84]:
len(_train)

4070

In [93]:
network = Network([3,6,2], func_model)

In [96]:
import datetime

In [110]:
datetime.datetime.now()

2017-05-14 01:48:30.296021


In [112]:
_start =datetime.datetime.now()
network.SGD(_train[:3000],10,100,0.01,test_data=_train[3000:])
_end=datetime.datetime.now()
print("time_consume: {}".format(_end-_start))

Epoch 0 Cost: 7.072403786179067
Epoch 1 Cost: 7.07233651746829
Epoch 2 Cost: 7.072271743890527
Epoch 3 Cost: 7.0722180048853245
Epoch 4 Cost: 7.072161711151477
Epoch 5 Cost: 7.072117996753177
Epoch 6 Cost: 7.072057307615895
Epoch 7 Cost: 7.072010253072393
Epoch 8 Cost: 7.071970273029936
Epoch 9 Cost: 7.071929777996506
time_consume: 0:00:02.292379
